In [10]:
import json
import pandas as pd
import os
import nf_model

In [8]:
extract_folder = "/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data"
json_path = os.path.join(extract_folder, "examples-test-original 2.json")

with open(json_path, 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)
print(df.columns)  # Spaltennamen
print(df.iloc[0])  # Erste Zeile

Index(['bass_synthetic_068-049-025', 'keyboard_electronic_001-021-127',
       'guitar_acoustic_010-066-100', 'reed_acoustic_037-068-127',
       'flute_acoustic_002-077-100', 'string_acoustic_056-047-075',
       'vocal_synthetic_003-088-025', 'brass_acoustic_046-101-050',
       'guitar_acoustic_014-070-050', 'string_acoustic_014-063-050',
       ...
       'mallet_acoustic_062-072-075', 'organ_electronic_028-067-100',
       'bass_synthetic_098-055-075', 'keyboard_electronic_069-089-100',
       'keyboard_electronic_069-054-127', 'brass_acoustic_006-088-075',
       'organ_electronic_028-044-100', 'string_acoustic_071-027-127',
       'string_acoustic_057-063-127', 'bass_synthetic_009-058-025'],
      dtype='object', length=4096)
bass_synthetic_068-049-025         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
keyboard_electronic_001-021-127    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
guitar_acoustic_010-066-100        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
reed_acoustic_037-068-127          [0, 0, 0, 0, 0, 0, 0, 0,

## Explaining what each column in the data menas


| Index | Qualität    | Beschreibung                        |
|-------|-------------|-----------------------------------|
| 0     | Bright      | Klang ist hell, klar               |
| 1     | Dark        | Klang ist dunkel, dumpf            |
| 2     | Distorted   | Klang ist verzerrt                 |
| 3     | Noisy       | Klang ist verrauscht               |
| 4     | Percussive  | Klang ist perkussiv (schlagzeugartig) |
| 5     | Sustained   | Klang ist gehalten, lang           |
| 6     | Harmonic    | Klang enthält viele Obertöne       |
| 7     | Metallic    | Klang klingt metallisch            |
| 8     | Vocal       | Klang klingt wie Stimme            |
| 9     | Other       | Andere unspezifizierte Eigenschaft |

In [11]:
input_data = nsynth["loudness"].values  # oder eine andere Spalte mit float-Werten
model = NormalizingFlow(input_data)
loss = model.training_step()
print("Loss:", loss)

NameError: name 'nsynth' is not defined